# 1.  **Título del Tema**


**Testing de Integración Ligera con `pytest` y `requests-mock`**

# 2.  **Explicación Conceptual Detallada**


*   **Definición y Propósito:** Las pruebas de integración ligera verifican que dos o más componentes de software (módulos, funciones, clases) colaboran correctamente. El objetivo no es probar la lógica interna de cada componente (eso es para las pruebas unitarias), sino probar el "contrato" o la "interfaz" entre ellos. ¿La función A le pasa los datos a la función B en el formato que B espera? ¿Qué pasa si la función A falla? ¿La función B maneja ese error correctamente?

*   **¿Cuándo y por qué se utiliza?:**
    *   **Cuando una función depende de otra:** Si `funcion_A()` llama a `funcion_B()`, una prueba de integración puede verificar esta interacción.
    *   **Cuando tu código interactúa con servicios externos:** Bases de datos, APIs web, sistemas de archivos, etc. En lugar de conectar con el servicio real (que puede ser lento, inestable o tener costos), "simulamos" su comportamiento con un **mock**.
    *   **Importancia:** Estas pruebas son cruciales porque muchos errores no ocurren dentro de una función aislada, sino en la comunicación entre ellas.

*   **Conceptos Clave y Herramientas:**
    *   **`pytest`:** Es nuestro marco de pruebas. Nos proporciona la estructura para escribir y ejecutar las pruebas de forma sencilla y potente (con `assert`, fixtures, etc.).
    *   **Mock (o Doble de Prueba):** Un "mock" es un objeto falso que simula el comportamiento de un objeto real de forma controlada. Para las pruebas de integración con servicios web, en lugar de hacer una llamada HTTP real, usamos un mock que finge ser el servidor y nos devuelve una respuesta predefinida (un éxito, un error 404, etc.).
    *   **`requests-mock`:** Es una biblioteca especializada en *mockear* (simular) las respuestas de la biblioteca `requests`. Es perfecta para probar cómo tu código maneja las respuestas de una API externa sin tener que conectarte realmente a ella.

*   **Ventajas:**
    1.  **Rapidez:** No dependen de la red ni de servicios externos, por lo que se ejecutan casi instantáneamente.
    2.  **Fiabilidad:** Los tests no fallarán porque la API externa esté caída o tu conexión a internet falle.
    3.  **Control Total:** Puedes simular cualquier escenario, incluyendo errores difíciles de replicar (ej: un error 503 del servidor) para asegurar que tu código es robusto.
    4.  **Aislamiento:** Si la prueba falla, sabes que el problema está en la interacción entre los componentes que estás probando, no en un sistema externo.

*   **Buenas Prácticas:**
    *   **Prueba el "contrato":** Enfócate en la comunicación. ¿Se llamó a la función correcta? ¿Con los argumentos correctos? ¿Se manejó bien la respuesta?
    *   **Nombres de prueba descriptivos:** `test_funcion_principal_cuando_api_devuelve_error_404()`.
    *   **No mockees lo que no es tuyo (si no es necesario):** No necesitas probar que la biblioteca `requests` funciona. Mockea la *respuesta* que `requests` te daría.


# 3.  **Sintaxis y Ejemplos Básicos**


**Sintaxis básica de `pytest`:**

Una prueba es simplemente una función que empieza con `test_`. Usamos `assert` para verificar que una condición es verdadera.

In [1]:
def test_suma_simple():
    assert 1 + 1 == 2

**Sintaxis básica de `requests-mock`:**

`requests-mock` se integra con `pytest` a través de una "fixture". Una fixture es como un asistente que prepara algo para tu prueba. La fixture se llama `requests_mock`.

In [4]:
import requests

# Imagina que esta es una función en tu código
def obtener_info_sitio_web():
    response = requests.get("http://ejemplo.com/api")
    return response.json()

# Así se vería una prueba básica con el mock
def test_obtener_info_con_mock(requests_mock):
    # 1. Configurar el mock:
    # Cuando se haga un GET a esta URL...
    requests_mock.get("http://ejemplo.com/api",
                      # ...devuelve este JSON con un status 200 (OK).
                      json={"mensaje": "Hola desde el mock"},
                      status_code=200)

    # 2. Ejecutar la función que queremos probar
    resultado = obtener_info_sitio_web()

    # 3. Verificar (Assert)
    # ¿La función procesó correctamente la respuesta del mock?
    assert resultado["mensaje"] == "Hola desde el mock"

# 4.  **Documentación y Recursos Clave**


*   **Documentación Oficial:**
    *   **pytest:** [https://docs.pytest.org/en/stable/](https://docs.pytest.org/en/stable/)
    *   **requests-mock:** [https://requests-mock.readthedocs.io/](https://requests-mock.readthedocs.io/)

*   **Recurso Adicional de Alta Calidad:**
    *   **Real Python - Mocking External APIs in Python:** [https://realpython.com/testing-third-party-apis-with-mock-servers/](https://realpython.com/testing-third-party-apis-with-mock-servers/) (Un excelente artículo que cubre conceptos similares con otras herramientas, muy enriquecedor).

# 5.  **Ejemplos de Código Prácticos**


**Paso 1: Crear nuestros módulos de código**

Vamos a crear dos módulos:
1.  `api_client.py`: Responsable de hablar con una API externa para obtener datos de un usuario.
2.  `user_service.py`: Usa el `api_client` para obtener datos y luego formatea un saludo para el usuario.


In [5]:
%%writefile api_client.py
# Este módulo simula la capa que interactúa directamente con una API externa.

import requests

def get_user_data(user_id):
    """
    Obtiene los datos de un usuario desde una API externa.
    Lanza una excepción si el usuario no se encuentra.
    """
    api_url = f"https://api.ejemplo.com/users/{user_id}"
    response = requests.get(api_url)

    # Si la respuesta es 404 (Not Found), lanzamos un error claro.
    if response.status_code == 404:
        raise ValueError(f"Usuario con ID {user_id} no encontrado.")

    # Si hubo otro tipo de error HTTP, requests lo lanzará aquí.
    response.raise_for_status()

    return response.json()

Writing api_client.py


In [6]:
%%writefile user_service.py
# Este módulo contiene la lógica de negocio.
# Depende de 'api_client' para obtener los datos.

from api_client import get_user_data

def generate_user_greeting(user_id):
    """
    Genera un saludo personalizado para un usuario.
    Integra la llamada a la API con la lógica de formato.
    """
    try:
        # Aquí está la integración: este módulo llama al otro.
        user_data = get_user_data(user_id)
        name = user_data.get("name")

        if not name:
            return "Hola, usuario anónimo."

        return f"¡Bienvenido, {name}!"
    except ValueError as e:
        # Captura el error específico de usuario no encontrado y lo maneja.
        return str(e)
    except Exception:
        # Captura cualquier otro error de la API (ej. 500 Internal Server Error)
        return "No se pudo contactar el servicio de usuarios en este momento."


Writing user_service.py


**Paso 2: Crear nuestro archivo de pruebas de integración**

Ahora, creamos la prueba que verifica que `user_service` y `api_client` funcionan bien juntos.

In [9]:
%%writefile test_integration_user_service.py

import pytest
from user_service import generate_user_greeting # La función que queremos probar

# Ejemplo 1: Prueba del "camino feliz" (todo funciona)
def test_greeting_for_existing_user(requests_mock):
    """
    Prueba la integración cuando la API devuelve un usuario válido.
    """
    user_id = 1
    # Mockeamos la respuesta que esperamos de la API
    requests_mock.get(
        f"https://api.ejemplo.com/users/{user_id}",
        json={"id": user_id, "name": "Ana", "email": "ana@ejemplo.com"},
        status_code=200
    )

    # Llamamos a la función de alto nivel
    greeting = generate_user_greeting(user_id)

    # Verificamos que el resultado final es el esperado
    assert greeting == "¡Bienvenido, Ana!"
    print("\n[ÉXITO] test_greeting_for_existing_user pasó.")


# Ejemplo 2: Prueba de un caso de error (usuario no encontrado)
def test_greeting_for_nonexistent_user(requests_mock):
    """
    Prueba la integración cuando la API devuelve un error 404.
    """
    user_id = 999
    # Mockeamos una respuesta de error 404 Not Found
    requests_mock.get(
        f"https://api.ejemplo.com/users/{user_id}",
        status_code=404
    )

    # Llamamos a la función de alto nivel
    greeting = generate_user_greeting(user_id)

    # Verificamos que nuestra función maneja el error correctamente
    assert greeting == f"Usuario con ID {user_id} no encontrado."
    print("[ÉXITO] test_greeting_for_nonexistent_user pasó.")


# Ejemplo 3: Prueba de un fallo genérico del servidor
def test_greeting_when_api_server_fails(requests_mock):
    """
    Prueba la integración cuando la API devuelve un error 500.
    """
    user_id = 2
    # Mockeamos una respuesta de error 500 Internal Server Error
    requests_mock.get(
        f"https://api.ejemplo.com/users/{user_id}",
        status_code=500
    )

    # Llamamos a la función de alto nivel
    greeting = generate_user_greeting(user_id)

    # Verificamos que se muestra el mensaje de error genérico
    assert greeting == "No se pudo contactar el servicio de usuarios en este momento."
    print("[ÉXITO] test_greeting_when_api_server_fails pasó.")

Overwriting test_integration_user_service.py


**Paso 3: Ejecutar las pruebas con `pytest`**

Ahora, le pedimos a `pytest` que ejecute las pruebas que acabamos de escribir en el archivo.

In [1]:
# -q es para modo "quiet" (menos verboso)
# El nombre del archivo le dice a pytest qué probar.
!pytest -q test_integration_user_service.py

# Salida esperada:
# [ÉXITO] test_greeting_for_existing_user pasó.
# [ÉXITO] test_greeting_for_nonexistent_user pasó.
# [ÉXITO] test_greeting_when_api_server_fails pasó.
# ...
# 3 passed in ...s

"pytest" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


# 6.  **Ejercicio Práctico**


# 7.  **Conexión con Otros Temas**


*   **Conceptos Previos:**
    *   **Pruebas Unitarias:** Deberías sentirte cómodo probando una función de forma aislada. La integración es el siguiente nivel.
    *   **Manejo de Excepciones (`try...except`):** Es fundamental para poder manejar los errores que pueden ocurrir en la comunicación entre componentes (como vimos en el ejemplo del 404).
    *   **APIs y HTTP:** Entender qué son las peticiones `GET` y los códigos de estado (200, 404, 500) es clave para saber qué simular.

*   **Temas Futuros:**
    *   **Pruebas End-to-End (E2E):** Después de las pruebas de integración, el siguiente paso es probar la aplicación completa, con una base de datos real y, a veces, incluso APIs reales (en un entorno de staging).
    *   **Integración Continua (CI):** Estas pruebas de integración son perfectas para ejecutarse automáticamente en un pipeline de CI/CD (como GitHub Actions) cada vez que subes nuevo código, asegurando que no has roto ninguna integración existente.
    *   **Mocking Avanzado (`unittest.mock`):** `requests-mock` es específico para `requests`. Para simular cualquier objeto o función en Python (no solo llamadas HTTP), usarás la biblioteca estándar `unittest.mock`.


# 8.  **Aplicaciones en el Mundo Real**


1.  **Microservicios:** En una arquitectura de microservicios, un servicio A a menudo necesita llamar a la API de un servicio B. Las pruebas de integración ligera son perfectas para probar que el servicio A maneja correctamente las respuestas (tanto de éxito como de error) del servicio B, sin necesidad de levantar toda la infraestructura de microservicios para la prueba.

2.  **Integración con Pasarelas de Pago (Stripe, PayPal):** Cuando desarrollas un e-commerce, necesitas probar tu lógica de "crear pago", "confirmar pago" y "manejar pago fallido". En lugar de hacer transacciones reales con tu tarjeta de crédito para cada prueba, mockeas la API de Stripe para que devuelva las respuestas que necesitas y así probar que tu sistema actualiza el estado del pedido correctamente en cada caso.